In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, LeaveOneOut, KFold
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("E:/research/Spacematch/formal research/feedback_89.csv") #import the data-points

In [3]:
df.head(5)

,user_id,place_id,sensations_temperature,preferences_temperature,clothing_level,indoor_temp,indoor_humidity,outdoor_temp,outdoor_humidity,created_at,Time
0,89,8939,4,2,1,76.86,39.82,64.94,84.273097,9/27/2021 14:33,10
1,89,8939,4,2,1,76.95,39.81,64.94,84.273097,9/27/2021 14:39,10
2,89,8921,3,3,2,69.39,48.94,64.94,84.273097,9/27/2021 14:51,10
3,89,8921,3,3,2,69.48,48.90,64.94,84.273097,9/27/2021 14:57,10
4,89,8935,4,2,2,75.27,40.21,64.94,84.273097,9/27/2021 15:08,11


In [4]:
y = df.sensations_temperature
X = df.drop(["user_id","place_id", "created_at","sensations_temperature","preferences_temperature"], axis = 1)  #Get X and y
# print(X.count())
rt_data = pd.read_csv("E:/research/Spacematch/formal research/real_time_room_data.csv")
rt_data = rt_data.drop(["room_id"], axis = 1)
X = X.append(rt_data)

In [5]:
X

,clothing_level,indoor_temp,indoor_humidity,outdoor_temp,outdoor_humidity,Time
0,1,76.860000,39.820000,64.94,84.273097,10
1,1,76.950000,39.810000,64.94,84.273097,10
2,2,69.390000,48.940000,64.94,84.273097,10
3,2,69.480000,48.900000,64.94,84.273097,10
4,2,75.270000,40.210000,64.94,84.273097,11
...,...,...,...,...,...,...
73,1,75.452974,41.623543,66.02,100.000000,10
0,1,69.030000,49.550000,88.00,55.000000,14
1,1,70.080000,48.020000,88.00,55.000000,14
2,1,73.710000,46.400000,88.00,55.000000,14


In [6]:
def createNewTime(row):
    if row['Time'] <= 12:
        return 1
    else:
        return 2
    
X['New Time'] = X.apply(createNewTime, axis=1)
X = X.drop(['Time'], axis = 1)
min_max_scaler = preprocessing.MinMaxScaler()
X = pd.DataFrame(min_max_scaler.fit_transform(X), columns = X.columns, index = X.index)
rt_data = X.iloc[-4:]
X = X.iloc[:74]

In [7]:
X

,clothing_level,indoor_temp,indoor_humidity,outdoor_temp,outdoor_humidity,New Time
0,0.0,0.838791,0.118141,0.000000,0.700871,0.0
1,0.0,0.847974,0.117332,0.000000,0.700871,0.0
2,1.0,0.076618,0.856118,0.000000,0.700871,0.0
3,1.0,0.085800,0.852881,0.000000,0.700871,0.0
4,1.0,0.676561,0.149699,0.000000,0.700871,0.0
...,...,...,...,...,...,...
69,0.0,0.031213,1.000000,0.046834,0.929913,0.0
70,0.0,0.060598,0.972721,0.046834,0.929913,0.0
71,0.0,0.610752,0.286956,0.046834,0.929913,0.0
72,0.0,0.658499,0.278344,0.046834,1.000000,0.0


In [8]:
param_test = {"n_neighbors"     :range(2,10,1),
              "weights"     :['uniform', 'distance'], 
              'p': (1, 2),
              'metric': ('minkowski', 'chebyshev'),
                  } # Optimize the parameters

cv_param = []
n_neighbors_param = []
weights_param = []
p_param = []
metric_param = []
accuracy_param = []
loo_accuracy_param = []

room_1_neutral = []
room_2_neutral = []
room_3_neutral = []
room_4_neutral = []


for i in range (2,9,1):
    gsearch = GridSearchCV(estimator = KNeighborsClassifier(), 
            param_grid = param_test, cv = i)
    gsearch.fit(X, y)
    n_neighbors_p   = gsearch.best_params_["n_neighbors"]
    weights_p       = gsearch.best_params_["weights"]
    p_p             = gsearch.best_params_["p"]
    metric_p        = gsearch.best_params_['metric']
    cv_param.append(i)
    n_neighbors_param.append(n_neighbors_p)
    weights_param.append(weights_p)
    p_param.append(p_p)
    metric_param.append(metric_p)
    accuracy_param.append(gsearch.best_score_)
    
    loo = LeaveOneOut() # test the accuracy with leaveoneout
    y_true = []
    y_pred = []
    for train_index, test_index in loo.split(X): # Split in X
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        knn = KNeighborsClassifier(n_neighbors = n_neighbors_p, weights = weights_p,
                                   metric = metric_p, p = p_p)
        knn = knn.fit(X_train, y_train) # train model
        predict_y = knn.predict(X_test)
        y_true.append(y_test[0])
        y_pred.append(predict_y[0])
    accuracy = accuracy_score(y_true, y_pred)
    loo_accuracy_param.append(accuracy
                             )
    knn = KNeighborsClassifier(n_neighbors = n_neighbors_p, weights = weights_p,
                                   metric = metric_p, p = p_p)
    knn = knn.fit(X, y)
    
    
    rt_prepro = knn.predict_proba(rt_data)
    room_1_neutral.append(rt_prepro[0][1])
    room_2_neutral.append(rt_prepro[1][1])
    room_3_neutral.append(rt_prepro[2][1])
    room_4_neutral.append(rt_prepro[3][1])

result = pd.DataFrame(list(zip(cv_param, n_neighbors_param,weights_param, p_param,metric_param, accuracy_param, loo_accuracy_param, 
                               room_1_neutral, room_2_neutral, room_3_neutral, room_4_neutral)),
                      columns = ['cv', 'N', 'weight', 'p', 'metric', 'accuracy', 'loo_accuracy', 
                                 'room_1', 'room_2', 'room_3', 'room_4'])

result.to_csv('E:/research/Spacematch/formal research/feedback_89_result_knn.csv')

In [9]:
# param_test = {"n_neighbors"     :range(2,10,1),
#               "weights"     :['uniform', 'distance'], 
#               'p': (1, 2),
#               'metric': ('minkowski', 'chebyshev'),
#               } # Optimize the parameters
# for i in range (2,9,1):
#     gsearch = GridSearchCV(estimator = KNeighborsClassifier(), 
#             param_grid = param_test, cv = i)
#     gsearch.fit(X, y)
#     print(i)
#     gsearch.fit(X, y)
#     n_neighbors_p     = gsearch.best_params_["n_neighbors"]
#     weights_p       = gsearch.best_params_["weights"]
#     p_p         = gsearch.best_params_["p"]
#     metric_p    = gsearch.best_params_['metric']
#     print(n_neighbors_p, weights_p, p_p, metric_p, gsearch.best_score_)
#     loo = LeaveOneOut() # test the accuracy with leaveoneout
#     y_true = []
#     y_pred = []
#     for train_index, test_index in loo.split(X): # Split in X
#         X_train, X_test = X.values[train_index], X.values[test_index]
#         y_train, y_test = y.values[train_index], y.values[test_index]
#         knn = KNeighborsClassifier(n_neighbors = n_neighbors_p, weights = weights_p,
#                                    metric = metric_p, p = p_p)
#         knn = knn.fit(X_train, y_train) # train model
#         predict_y = knn.predict(X_test)
#         y_true.append(y_test[0])
#         y_pred.append(predict_y[0])
#     accuracy = accuracy_score(y_true, y_pred)
#     print("accuracy is:", accuracy)
#     knn = KNeighborsClassifier(n_neighbors = n_neighbors_p, weights = weights_p,
#                                    metric = metric_p, p = p_p)
#     knn = knn.fit(X, y)
#     print(knn.predict(rt_data))
#     rt_prepro = knn.predict_proba(rt_data)
#     print("probability : \n", rt_prepro)
# print(rt_data)